In [5]:
#케라스 API로 데이터셋 불러오기
from tensorflow import keras

(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()

In [6]:
#인공신경망 모델 데이터 준비
from sklearn.model_selection import train_test_split

train_scaled = train_input / 255.0      #이미지의 픽셀값을 0~1사이의 값으로 변환
train_scaled = train_scaled.reshape(-1, 28*28)      #2차원 이미지 배열을 1차열 배열로 변환
train_scaled, val_scaled, train_target, val_target = train_test_split(train_scaled, train_target, test_size=0.2, random_state=42)

In [7]:
#밀집층 추가.
dense1 = keras.layers.Dense(100, activation='sigmoid', input_shape=(784,))      #은닉층. (784,)크기의 입력 데이터를 받아 100개의 뉴런에 연결하고 활성화 함수는 시그모이드 함수를 사용한다.
dense2 = keras.layers.Dense(10, activation='softmax')     #출력층. 10개의 뉴런으로 연결하고 활성화 함수는 softmax함수를 사용한다. 입력 크기는 첫 레이어 에서만 정해준다.

In [9]:
#심층 신경망 제작
model = keras.Sequential([dense1, dense2])      #[이미지 배열] -> [입력층] -> [은닉층] -시그모이드 함수-> [출력층] -소프트맥스 함수-> [확률]

In [10]:
#모델 구조 정보 출력
model.summary()     #summary() 매서드를 호출하면 모델의 이름. 모델에 들어있는 층, 층 이름, 클래스, 출력 크기, 모델 파라미터 개수가 출력된다.
##층을 정의 할 때 name 매개변수를 통해 이름을 지정할 수 있다. 미지정시 자동으로 dense라 명명한다.

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               78500     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1010      
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


출력 크기를 보면 샘플의 개수를 나타내는 첫번째 차원값이 None이다. 이는 샘플의 개수가 아직 정의 되어있지 않기 때문인데, 케라스 모델의 fit() 메서드는 미니배치 경사 하강법을 이용하기 때문이다.<br/>
미니배치 크기의 기본값은 32이며, 이 값은 fit() 메서드에서 batch_size 매개변수를 통해 바꿀 수 있다. 따라서 초기에는 None으로 설정되어있는 것이다.<br/>
이렇게 신경망 층에 입력되거나 출력되는 배열의 첫 번째 차원을 __배치 차원__이라고 부른다.<br/>
모델 파라미터의 개수는 $$가중치\ 개수 + 절편\ 개수 = [입력\ 뉴런\ 개수 * 출력\ 뉴런\ 개수] + 출력\ 뉴런\ 개수$$ 로 구할 수 있다.<br/>
간혹 경사 하강법으로 훈련되지 않는 파라미터를 가진 층이 있는데 이런 층의 파라미터 개수가 Non-trainable params에 출력된다.

In [12]:
#Sequential 클래스 생성자 내에서 Dense 클래스 객체 만들기
model = keras.Sequential([keras.layers.Dense(100, activation='sigmoid', input_shape=(784,), name='hidden'), 
                          keras.layers.Dense(10, activation='softmax', name='output')], name='패션 MNIST 모델')     #모델이름은 한글이 들어가도 되지만, 층의 이름은 영문이어야 함
model.summary()
#밀집층을 따로 정의했을 때보다 추가되는 층에 대한 정보를 한눈에 알 수 있다는 장점이 있다. 다만 Sequential 클래스 생성자가 매우 길어지고 조건에 따라 층을 추가할 수 없다.

Model: "패션 MNIST 모델"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hidden (Dense)               (None, 100)               78500     
_________________________________________________________________
output (Dense)               (None, 10)                1010      
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


In [15]:
#add()매서드를 통해 층을 추가하기
model = keras.Sequential()
model.add(keras.layers.Dense(100, activation='sigmoid', input_shape=(784,), name='hidden'))
model.add(keras.layers.Dense(10, activation='softmax', name='output'))
model.summary()
#추가되는 층을 한눈에 파악할 수 있고 프로그램 실행 시 동적으로 층을 선택하여 추가할 수 있다.

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hidden (Dense)               (None, 100)               78500     
_________________________________________________________________
output (Dense)               (None, 10)                1010      
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


In [16]:
#모델 훈련
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')
model.fit(train_scaled, train_target, epochs=5)     #추가한 층이 성능을 향상시켰음을 확인할 수 있다.

Epoch 1/5
1500/1500 [==============================] - 4s 2ms/step - loss: 0.5650 - accuracy: 0.8070
Epoch 2/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.4071 - accuracy: 0.8532
Epoch 3/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3721 - accuracy: 0.8659
Epoch 4/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3494 - accuracy: 0.8736
Epoch 5/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3323 - accuracy: 0.8800


위의 모델에 은닉층에 시그모이드 함수를 사용했는데, 시그모이드 함수는 절댓값이 커질수록 기울기가 작아지기 때문에 올바른 출력을 만드는데 신속하게 대응하지 못한다.<br/> 특히 층이 많은 심층 신경망일수록 그 부작용이 누적되어 학습을 더 어렵게 만든다. 이를 개선하기 위에 제안된 활성화 함수가 바로 __렐루(ReLU) 함수__ 이다.<br/>
렐루 함수는 max(0, z)와 같이 쓸 수 있고 z가 0보다 크면 z를 출력하고, z가 0보다 작으면 0을 출력한다. 렐루 함수는 특히 이미지 처리에서 좋은 성능을 낸다.

In [17]:
#렐루 함수를 이용한 모델
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28, 28)))     #Flatten 클래스는 배치 차원을 제외하고 나머지 입력 차원을 모두 1차원으로 펼친다.
model.add(keras.layers.Dense(100, activation='ReLU', name='hidden'))
model.add(keras.layers.Dense(10, activation='softmax', name='output'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
hidden (Dense)               (None, 100)               78500     
_________________________________________________________________
output (Dense)               (None, 10)                1010      
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


In [18]:
#훈련 데이터 준비
(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()
train_scaled = train_input / 255.0      #이미지의 픽셀값을 0~1사이의 값으로 변환. flatten 층을 이용하기 때문에 reshape는 사용할 필요가 없다.
train_scaled, val_scaled, train_target, val_target = train_test_split(train_scaled, train_target, test_size=0.2, random_state=42)

In [19]:
#모델 훈련
model.compile(loss='sparse_categorical_crossentropy', metrics='accuracy')
model.fit(train_scaled, train_target, epochs=5)     #시그모이드 함수보다 성능이 향상됨.

Epoch 1/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.5418 - accuracy: 0.8094
Epoch 2/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3963 - accuracy: 0.8578
Epoch 3/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3585 - accuracy: 0.8707
Epoch 4/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3373 - accuracy: 0.8787
Epoch 5/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3202 - accuracy: 0.8862


In [20]:
#검증 세트에서의 성능 확인
model.evaluate(val_scaled, val_target)

375/375 [==============================] - 1s 1ms/step - loss: 0.3922 - accuracy: 0.8666


[0.3922073245048523, 0.8665833473205566]

__옵티마이저(optimizer)__ : 경사하강법 알고리즘의 총칭


> SGD 옵티마이저 : 기본적인 확률적 경사 하강법 알고리즘<br/>
Adagrad, RMSprop : 적응적 학습률을 이용하는 알고리즘<br/>
Adam : 모멘텀 최적화와 RMSprop의 장점을 접목한 알고리즘



In [25]:
#Adam 옵티마이저를 이용하는 모델
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28, 28)))
model.add(keras.layers.Dense(100, activation='ReLU', name='hidden'))
model.add(keras.layers.Dense(10, activation='softmax', name='output'))

In [28]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics='accuracy')
model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2920 - accuracy: 0.8918
Epoch 2/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2774 - accuracy: 0.8976
Epoch 3/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2663 - accuracy: 0.9010
Epoch 4/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2545 - accuracy: 0.9056
Epoch 5/5
1500/1500 [==============================] - 3s 2ms/step - loss: 0.2445 - accuracy: 0.9083


In [29]:
model.evaluate(val_scaled, val_target)

375/375 [==============================] - 1s 1ms/step - loss: 0.3298 - accuracy: 0.8851


[0.3298027217388153, 0.8850833177566528]